### This notebook explores the different features of pyproj library

In [13]:
from shapely.geometry import box
from pyproj import database, aoi
import geopandas as gpd
from lonboard import Map, PolygonLayer
from lonboard.basemap import CartoBasemap

In [14]:
def search_crs_by_area(area_name, object_type="all", filter_text=None):
    """
    Searches for CRS applicable to a given area.

    Parameters
    ----------
    area_name : str or pyproj.aoi.AreaOfInterest
        The name of the area or AreaOfInterest object to search for.
    object_type : str, optional
        The type of CRS or other database object to search for.
        Can be "all", "geographic", "projected", or other CRS types supported by pyproj.
        Defaults to "all".
    filter_text : str, optional
        A string of text to filter the results by, performing a case-insensitive
        search on the `area_of_use` attribute of each CRS. Defaults to None.

    Returns
    -------
    list
        A list of CRS objects that are applicable to the given area and CRS type.
    """

    if object_type == "all":
        crs_list = database.query_crs_info(area_of_interest=area_name, contains=False)
    else:
        crs_list = database.query_crs_info(
            area_of_interest=area_name, pj_types=object_type.upper(), contains=False
        )

    if filter_text:
        crs_list = [
            crs
            for crs in crs_list
            if filter_text.lower() in crs.area_of_use.name.lower()
        ]

    return crs_list


In [15]:
argentina_bbox = (
    -73.582520,
    -55.162286,
    -21.781349,
    -53.637451
)  # (minx, miny, maxx, maxy)

In [16]:
argentina_aoi = aoi.AreaOfInterest(
    west_lon_degree=argentina_bbox[0],
    south_lat_degree=argentina_bbox[3],
    east_lon_degree=argentina_bbox[1],
    north_lat_degree=argentina_bbox[2]
)

In [17]:
argentiana_crs_list = search_crs_by_area(
    area_name=argentina_aoi,
    object_type="GEOGRAPHIC_2D_CRS",
    filter_text="Argentina"
)

In [18]:
for crs in argentiana_crs_list:
    print(f"EPSG:{crs.code} - {crs.name} - Type: {crs.type}")
    print(f"Area of Use: {crs.area_of_use.bounds}")

EPSG:4160 - Chos Malal 1914 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-72.14, -43.41, -65.86, -31.91)
EPSG:4161 - Pampa del Castillo - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.59, -50.34, -65.47, -42.49)
EPSG:4190 - POSGAR 98 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.59, -58.41, -52.63, -21.78)
EPSG:4221 - Campo Inchauspe - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.59, -54.93, -53.65, -21.78)
EPSG:4254 - Hito XVIII 1963 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-74.83, -55.96, -63.73, -51.65)
EPSG:4326 - WGS 84 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-180.0, -90.0, 180.0, 90.0)
EPSG:4694 - POSGAR 94 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.59, -58.41, -52.63, -21.78)
EPSG:5340 - POSGAR 2007 - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.59, -58.41, -52.63, -21.78)
EPSG:9248 - Tapi Aike - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-73.28, -52.43, -68.3, -50.33)
EPSG:9251 - MMN - Type: PJType.GEOGRAPHIC_2D_CRS
Area of Use: (-68.64,

In [19]:
# Create a list of dictionaries to store CRS information and geometry
crs_data = []

for crs in argentiana_crs_list:
    # pyproj.AreaOfUse.bounds is (west, south, east, north)
    # shapely.geometry.box is (minx, miny, maxx, maxy)
    if int(crs.code) == 4326:
        # Skip WGS84 as it covers the whole world
        continue
    bounds = crs.area_of_use.bounds
    crs_data.append(
        {
            "name": crs.name,
            "epsg_code": crs.code,
            "geometry": box(*bounds),
        }
    )

# Create a GeoDataFrame from the list of dictionaries
gdf_aoi = gpd.GeoDataFrame(crs_data, crs="EPSG:4326")

# Sort the GeoDataFrame by area in descending order so smaller polygons are plotted on top
gdf_aoi = gdf_aoi.reindex(gdf_aoi.geometry.area.sort_values(ascending=False).index)

C:\Users\learo\AppData\Local\Temp\ipykernel_9732\1267083373.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_aoi = gdf_aoi.reindex(gdf_aoi.geometry.area.sort_values(ascending=False).index)


In [20]:
def hexcolor_to_rgb(hex_color):
    """Convert hex color to RGB tuple."""
    hex_color = hex_color.lstrip('#')
    return [int(hex_color[i:i+2], 16) for i in (0, 2, 4)]

In [21]:
COLORS = [
    "#FC49A3",  # pink
    "#CC66FF",  # purple-ish
    "#66CCFF",  # sky blue
    "#66FFCC",  # teal
    "#00FF00",  # lime green
    "#FFCC66",  # light orange
    "#FF6666",  # salmon
    "#FF0000",  # red
    "#FF8000",  # orange
    "#FFFF66",  # yellow
    "#00FFFF",  # turquoise
]

In [31]:
# Create a lonboard map and add the GeoDataFrame as a layer
aoi_lyr = PolygonLayer.from_geopandas(
    gdf=gdf_aoi,
    get_fill_color='orange',
    opacity=0.5,
    get_line_color=[0, 0, 0],
    get_line_width=2,
    auto_highlight=True
)


Map(
    layers=[aoi_lyr],
    view_state={"longitude": -64, "latitude": -34, "zoom": 3},
    basemap_style=CartoBasemap.DarkMatter
)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…